In [87]:
import pandas as pd
import numpy as np

from pathlib import Path

In [88]:
DATA_PATH = Path("../data/silver/combined_data.csv")
df = pd.read_csv(DATA_PATH)

In [89]:
# Ensure year is int
df['year'] = df['year'].astype(int)

# Get feature columns (excluding RA and year)
feature_cols = [col for col in df.columns if col not in ['RA', 'year']]

# Find the last year for each RA
last_year_per_ra = df.groupby('RA')['year'].max()

In [90]:
# Create the gold dataframe
gold_data = []

for ra in df['RA'].unique():
    ra_df = df[df['RA'] == ra].sort_values('year')
    last_year = ra_df['year'].max()
    
    # Get the row with the last year
    last_year_data = ra_df[ra_df['year'] == last_year].iloc[0]
    
    row_dict = {'RA': ra}
    
    # Add current year features
    for col in feature_cols:
        row_dict[col] = last_year_data[col]
    
    # Add lagged features
    years_available = sorted(ra_df['year'].unique())
    for i, year in enumerate(years_available):
        year_data = ra_df[ra_df['year'] == year].iloc[0]
        has_lag = last_year - year
        if has_lag > 0:
            for col in feature_cols:
                row_dict[f"{col}__minus_{has_lag}"] = year_data[col]
    
    gold_data.append(row_dict)

df_gold = pd.DataFrame(gold_data)

In [91]:
# Reorder columns: RA first, then all current features, then all minus_1, then all minus_2, etc.
ordered_cols = ['RA']

# Add current year features
for col in feature_cols:
    ordered_cols.append(col)

# Add lagged features by lag level
lag = 1
while True:
    lag_cols = [f"{col}__minus_{lag}" for col in feature_cols if f"{col}__minus_{lag}" in df_gold.columns]
    if not lag_cols:
        break
    ordered_cols.extend(lag_cols)
    lag += 1

df_gold = df_gold[ordered_cols]

In [92]:
# Create a column with the number of year (years of data) for each RA
def count_years(row):
    # Check if minus_2 columns have data (3 years)
    minus_2_cols = [col for col in row.index if '__minus_2' in col]
    if minus_2_cols and pd.notna(row[minus_2_cols[0]]):
        return 3
    
    # Check if minus_1 columns have data (2 years)
    minus_1_cols = [col for col in row.index if '__minus_1' in col]
    if minus_1_cols and pd.notna(row[minus_1_cols[0]]):
        return 2
    
    # Only current year (1 year)
    return 1


In [93]:
df_gold['years'] = df_gold.apply(count_years, axis=1)

print(f"\nYears distribution:")
print(df_gold['years'].value_counts().sort_index())


Years distribution:
years
1    752
2    380
3    444
Name: count, dtype: int64


In [94]:
# Convert numeric columns to float (replace comma with dot and handle 'nan' strings)
import numpy as np

numeric_cols = [c for c in df_gold.columns if c not in ['RA', 'has_lag']]

for col in numeric_cols:
    # normalize strings and missing values
    df_gold[col] = (
        df_gold[col]
        .astype(str)
        .str.replace(',', '.', regex=False)
        .replace('nan', np.nan)
    )
    # attempt conversion to float where appropriate
    try:
        df_gold[col] = df_gold[col].astype(float)
    except Exception:
        # leave column as-is if conversion fails
        pass

# show snapshot
df_gold.head()

,RA,IAA,IEG,IPS,math,portuguese,has_lag,IAA__minus_1,IEG__minus_1,IPS__minus_1,math__minus_1,portuguese__minus_1,has_lag__minus_1,IAA__minus_2,IEG__minus_2,IPS__minus_2,math__minus_2,portuguese__minus_2,has_lag__minus_2,years
0,RA-1,8.3,4.1,5.6,2.7,3.5,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,RA-2,8.8,5.2,6.3,6.3,4.5,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,RA-3,0.0,7.9,5.6,5.8,4.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,RA-4,8.8,4.5,5.6,2.8,3.5,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,RA-5,7.9,8.6,5.6,7.0,2.9,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [95]:
df_gold.to_csv("../data/gold/gold_data.csv", index=False)